In [1]:
import duckdb
import pandas as pd
import numpy as np
import re
conn = duckdb.connect()
print(conn)
conn = duckdb.connect(database = 'db.duckdb',read_only = False)


In [2]:
path = "/workspaces/NSWEducationDataModelling/data/2022attendancefinal.csv"
con = duckdb.connect()
q1 = f""" select * from '{path}'  """
data = con.execute(q1).df()

data= data.dropna()

data = data.reset_index(drop = True)
print("After Removing Nan",data)

After Removing Nan       school_code                school_name  2011  2012  2013  2014  2015  \
0          1001.0   Abbotsford Public School  94.3  95.9  95.7  95.3  94.6   
1          1002.0     Aberdeen Public School  93.6  94.2  95.2  95.3    95   
2          1003.0     Abermain Public School  93.5    93    92  92.8  91.4   
3          1007.0    Adaminaby Public School  95.4  96.4  97.2  96.2  90.9   
4          1008.0    Adamstown Public School  94.8  95.4  94.7  95.8  95.1   
...           ...                        ...   ...   ...   ...   ...   ...   
2024       8155.0          Young High School  85.6  85.1  87.8  87.3  88.4   
2025       4131.0  Young North Public School  90.7  90.6  91.1    92  92.4   
2026       3563.0        Young Public School  94.1  93.4  93.6  94.1  94.2   
2027       4124.0    Yowie Bay Public School  96.4  95.7    96  96.3  96.1   
2028       3566.0      Zig Zag Public School    89  90.6  93.6  91.8  92.7   

      2016  2017  2018  2019  2021  2022  
0

In [3]:
data.dtypes

school_code    float64
school_name     object
2011            object
2012            object
2013            object
2014            object
2015            object
2016            object
2017            object
2018            object
2019            object
2021            object
2022            object
dtype: object

In [4]:
attendance_unpivot = pd.melt(data,id_vars=['school_code','school_name'],value_vars = ['2011','2012','2013','2014','2015','2016','2017','2018','2019','2021','2022'])

In [5]:
attendance_unpivot

,school_code,school_name,variable,value
0,1001.0,Abbotsford Public School,2011,94.3
1,1002.0,Aberdeen Public School,2011,93.6
2,1003.0,Abermain Public School,2011,93.5
3,1007.0,Adaminaby Public School,2011,95.4
4,1008.0,Adamstown Public School,2011,94.8
...,...,...,...,...
22314,8155.0,Young High School,2022,79.7
22315,4131.0,Young North Public School,2022,88
22316,3563.0,Young Public School,2022,86.9
22317,4124.0,Yowie Bay Public School,2022,89.7


In [6]:
attendance_unpivot.head(2)
# renaming columns 
attendance_unpivot.rename(columns = {'variable':'Year','value':'Attendance'},inplace = True)
attendance_unpivot=attendance_unpivot.dropna()

attendance_unpivot['school_code']=attendance_unpivot.school_code.astype(int)
attendance_unpivot['Year']=attendance_unpivot.Year.astype(int)
# attendance_unpivot['Attendance']=attendance_unpivot.Attendance.astype(float)
attendance_unpivot



,school_code,school_name,Year,Attendance
0,1001,Abbotsford Public School,2011,94.3
1,1002,Aberdeen Public School,2011,93.6
2,1003,Abermain Public School,2011,93.5
3,1007,Adaminaby Public School,2011,95.4
4,1008,Adamstown Public School,2011,94.8
...,...,...,...,...
22314,8155,Young High School,2022,79.7
22315,4131,Young North Public School,2022,88
22316,3563,Young Public School,2022,86.9
22317,4124,Yowie Bay Public School,2022,89.7


In [7]:
attendance_unpivot['Attendance'] = attendance_unpivot['Attendance'].replace('na', '')
attendance_unpivot['Attendance']=pd.to_numeric(attendance_unpivot['Attendance'], errors='coerce')
import csv
attendance_unpivot.to_csv('attendance_unpivot.csv')

attendance_unpivot_2011 = attendance_unpivot[attendance_unpivot['Year']==2011]


# gk = attendance_unpivot_2011.groupby('Year')
# gk.first()
attendance_unpivot_groupby=attendance_unpivot.groupby([ 'Year'])['Attendance'].max()

print(attendance_unpivot_groupby)

Year
2011    98.7
2012    98.6
2013    98.4
2014    98.8
2015    99.2
2016    98.1
2017    98.8
2018    97.5
2019    97.7
2021    97.4
2022    96.5
Name: Attendance, dtype: float64
